# Evaluación de los métodos de transcripción de audio a texto

In [7]:
#Lectura de la base de datos
import pandas as pd

#Transcriptores de audio
import azure.cognitiveservices.speech as speechsdk
import json
import wave
import os
from os.path import join, dirname
import speech_recognition as sr
import pandas as pd
from pydub import AudioSegment 
from pydub.silence import split_on_silence

#Calculo de la similaridad
import nltk 
from nltk.corpus import wordnet as wn
import numpy as np

In [8]:
#Lectura del fichero de datos CSV. Separación por ; e ignorando el espacio inicial
data = pd.read_csv('database_speech.csv', sep = ';', skipinitialspace = True)

#Eliminacion de fases
#data = data.drop(data[data['Phase']==8].index)

#Muestra de datos cargados (10)
data.head(10)

,URL,Video,Transcription,Phase,chunk
0,https://www.youtube.com/watch?v=amreUFtw6Ks,1,la primera incisión es de 10 milímetros en el ...,1,20
1,https://www.youtube.com/watch?v=amreUFtw6Ks,1,por ahí se mete la cámara.,1,21
2,https://www.youtube.com/watch?v=amreUFtw6Ks,1,luego se necesitan otras tres para los instrum...,1,22
3,https://www.youtube.com/watch?v=amreUFtw6Ks,1,una aquí de 10 milímetros y otras dos pequeñit...,1,23
4,https://www.youtube.com/watch?v=amreUFtw6Ks,1,"mira, ahí están metiendo uno de los trocares p...",1,9000
5,https://www.youtube.com/watch?v=amreUFtw6Ks,1,"eso lindo que está ahí rosadito, como rojito, ...",8,62
6,https://www.youtube.com/watch?v=amreUFtw6Ks,1,"eso que está cogiendo ahí con la pinza, esa es...",8,8000
7,https://www.youtube.com/watch?v=amreUFtw6Ks,1,ahí metieron una gasita para poder limpiar y p...,1,7000
8,https://www.youtube.com/watch?v=amreUFtw6Ks,1,ay espérate y verás que ahora te la sacan. ahí...,1,6000
9,https://www.youtube.com/watch?v=amreUFtw6Ks,1,"bueno, para que podamos ver todo lo que estamo...",1,404


In [9]:
#Microsoft Azure
##Claves de suscripción
speech_key, service_region = "3c08f04c3e6d4a15806ff522ef594cb7", "westeurope"
def speech_recognize_bing_azure(speech_key,service_region,filename):
    #Inicialización y configuración
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    speech_config.speech_recognition_language="es-ES"
    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    #Obtención de la transcripción
    result = speech_recognizer.recognize_once()
    #Comprobación de que el transcriptor ha obtenido resultado
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        transcript = result.text
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("No se pudo reconocer el audio: {}".format(result.no_match_details))
        transcript = "No hemos reconocido el audio"
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("El reconocimiento del audio fue cancelado: {}".format(cancellation_details.reason))
        transcript = "No hemos reconocido el audio"
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Detalles de error: {}".format(cancellation_details.error_details))
            transcript = "No hemos reconocido el audio"
    return transcript

#Sphinx
def speech_recognize_sphinx(audio,r):
    try:
        #Obtención de la transcripción
        transcript = r.recognize_sphinx(audio,language='es-es')
        #Comprobación de que el transcriptor ha obtenido resultado
    except sr.UnknownValueError:
        print("Sphinx no pudo transcribir el audio")
        transcript = "No hemos reconocido el audio"
    except sr.RequestError as e:
        print("Sphinx error; {0}".format(e))
        transcript = "No hemos reconocido el audio"
    except KeyboardInterrupt:
        print("Paramos para poder continuar")
        transcript = "No hemos reconocido el audio"
    return transcript

#Google
def speech_recognize_google(audio,r):
    try:
        #Obtención de la transcripción
        transcript = r.recognize_google(audio,language = 'es-es') 
        #Comprobación de que el transcriptor ha obtenido resultado
    except sr.UnknownValueError:
        print("Google Speech Recognition no pudo entender el audio")
        transcript = "No hemos reconocido el audio"
    except sr.RequestError as e:
        print("No se pudieron hacer peticiones de transcripción del audio a Google; {0}".format(e))
        transcript = "No hemos reconocido el audio"
    except KeyboardInterrupt:
        print("Paramos para poder continuar")
        transcript = "No hemos reconocido el audio"
    return transcript

#Wit
#Claves de la suscripción
WIT_AI_KEY = "OQCE7MPRRP42IBEDELTZOEODRDDIVZVC"
def speech_recognize_wit(audio, r, WIT_AI_KEY): 
    #Obtención de la transcripción
    try:
        transcript = r.recognize_wit(audio, key=WIT_AI_KEY)
        #Comprobación de que el transcriptor ha obtenido resultado
    except sr.UnknownValueError:
        print("Wit no pudo reconocer el audio")
        transcript = "No hemos reconocido el audio"
    except sr.RequestError as e:
        print("No se pudo hacer una peticiones a los servidores de Wit; {0}".format(e))
        transcript = "No hemos reconocido el audio"
    except KeyboardInterrupt:
        print("Paramos para poder continuar")
        transcript = "No hemos reconocido el audio"
    return transcript

#Get wav file from audio data
def get_wav_audio(audio,i):

    #Parametros del wav
    CHANNELS = 1
    RATE = 44100
    archivo = "grabacion" + str(i) + ".wav"

    waveFile = wave.open(archivo,'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(2)
    waveFile.setframerate(RATE)
    frame = audio.get_raw_data()
    waveFile.writeframes(frame)
    waveFile.close()
    filename = "/Users/marcosgt11/Desktop/CuartoGIB/TFG/" + archivo 
    return filename

In [10]:
#Conversión de la etiqueta dada por nltk a una etiqueta usada por wordnet.synsets
def convert_tag(tag):
    #Identifica la etiqueta de nltk y la transforma
    if tag.startswith("N"):
        converted_tag = wn.NOUN
    elif tag.startswith("J"):
        converted_tag = wn.ADJ
    elif tag.startswith("R"):
        converted_tag = wn.ADV
    elif tag.startswith("V"):
        converted_tag = wn.VERB
    else:
        converted_tag = None
    return converted_tag

#Obtención de los synsets de un documento
def doc_to_synsets(doc):
    #Tokeniza el documento
    tokens = nltk.word_tokenize(doc)
    #Obtiene sus etiquetas nltk
    pos_tags = nltk.pos_tag(tokens)
    match_list = []
    #Transforma las etiquetas y las almacena en una lista
    for i in range(0,len(tokens)):
        if convert_tag(pos_tags[i][1]) is not None:
            match = wn.synsets(pos_tags[i][0], convert_tag(pos_tags[i][1]))
            if match:
                match_list.append(match[0])
  
    return match_list

#Cálculo de la similaridad entre synsets de 2 frases diferentes
def similarity_score(s1, s2):
    score_final = []
    #Recorre las dos frases
    for syn1 in s1:
        score = []
        for syn2 in s2:
            #Calcula la similaridad y la almacena
            similarity = syn1.path_similarity(syn2)
            if similarity is not None:
                score.append(similarity)
        if score:
            score_final.append(max(score))
    #Finalmente normaliza el resultado de similaridad
    normalized = np.mean(score_final)
    return normalized

#Calcula la similaridad entre 2 documentos diferentes
def document_path_similarity(doc1, doc2):
    #Obtiene los synsets de cada documento
    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)
    #Calcula la similaridad entre los synsets obtenidos
    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

In [11]:
def analyze_audio(pathAudio):
    #Inicializa el recognizer y los arrays de transcripciones
    r = sr.Recognizer()
    pathName = os.path.dirname(pathAudio)
    i = 0
    sphinx = []
    google = []
    wit = []
    azure = [] 
    files = []
    
    #Lee todos los fragmentos
    for file in os.listdir(pathName): 
        print("Procesando fragmento "+str(i))
        #No procesa el fichero por defecto de audio generado en Mac
        if(file == ".DS_Store"):
            continue
        #Guarda el nombre del archivo de audio
        files.append(file)
        print(pathName+'/'+file)
        
        #Obtiene la transcripción de Azure
        azure.append(speech_recognize_bing_azure(speech_key,service_region,pathName+'/'+file))
        #Obtiene la transcripción de los otros 3 transcriptores
        with sr.AudioFile(pathName+'/'+file) as source:
            r.adjust_for_ambient_noise(source, duration = 0.5)
            audio = r.record(source)
            sphinx.append(speech_recognize_sphinx(audio,r))
            google.append(speech_recognize_google(audio,r))
            wit.append(speech_recognize_wit(audio,r,WIT_AI_KEY)) 
            i += 1   
            
    return (sphinx,google,wit,azure,files)

In [12]:
#Fichero de audio para analizar
pathAudio = '/Users/marcosgt11/Desktop/CuartoGIB/TFG/AudiosDEF/Audio12/'
#Llamada a la función
sphinx,google,wit,azure,files = analyze_audio(pathAudio)

#Lee los datos extraidos de las transcripciones automáticas
h={'sphinx':sphinx,'google':google,'wit':wit,'azure':azure,'filename':files} 
result = pd.DataFrame(h)

#Después lee las transcripciones manuales
df_transcript = pd.read_csv('/Users/marcosgt11/Desktop/CuartoGIB/TFG/database_speechA12.csv',sep=';')

#Inicializa los arrays donde se guarda la similaridad de la transcripcion con la hecha a mano
PS_sphinx = []
PS_google = []
PS_wit = []
PS_azure = []
tsphinx = []
tgoogle = []
twit = []
tazure = []
t = []

#Ordena ambos sets de datos y extrae sus indices
df_sorted = result.sort_values(by=['filename'])
df_sortedtr = df_transcript.sort_values(by=['chunk'])
indexesi = df_sorted.index
indexesj = df_sortedtr.index

#Recorre todas las posiciones de los sets de datos para ordenarlos
for i in range(len(indexesi)):
    
        PS_sphinx.append(document_path_similarity(df_sorted['sphinx'].loc[indexesi[i]],df_sortedtr['Transcription'].loc[indexesj[i]]))
        PS_google.append(document_path_similarity(df_sorted['google'].loc[indexesi[i]],df_sortedtr['Transcription'].loc[indexesj[i]]))
        PS_wit.append(document_path_similarity(df_sorted['wit'].loc[indexesi[i]],df_sortedtr['Transcription'].loc[indexesj[i]]))
        PS_azure.append(document_path_similarity(df_sorted['azure'].loc[indexesi[i]],df_sortedtr['Transcription'].loc[indexesj[i]]))
        tsphinx.append(df_sorted['sphinx'].loc[indexesi[i]])
        tgoogle.append(df_sorted['google'].loc[indexesi[i]])
        twit.append(df_sorted['wit'].loc[indexesi[i]])
        tazure.append(df_sorted['azure'].loc[indexesi[i]])
        t.append(df_sortedtr['Transcription'].loc[indexesj[i]])
        
#Guarda los resultados en un excel
similarities = {'sphinx':PS_sphinx,'google':PS_google,'wit':PS_wit,'azure':PS_azure, 'Transcripción sphinx': tsphinx, 'Transcripción google': tgoogle, 'Transcripción wit': twit,'Transcripción azure': tazure,'Transcripción a mano': t} 
similitud = pd.DataFrame(similarities)
#similitud.to_excel('/Users/marcosgt11/Desktop/CuartoGIB/TFG/all_transcriptA12.xls') 

Procesando fragmento 0
/Users/marcosgt11/Desktop/CuartoGIB/TFG/AudiosDEF/Audio12/.12_chunk12.wav.icloud


RuntimeError: Exception with an error code: 0xa (SPXERR_INVALID_HEADER)
[CALL STACK BEGIN]

3   libMicrosoft.CognitiveServices.Spee 0x000000011c40395a file_logger_stop_logging + 1493226
4   libMicrosoft.CognitiveServices.Spee 0x000000011c4030bb file_logger_stop_logging + 1491019
5   libMicrosoft.CognitiveServices.Spee 0x000000011c31efd3 file_logger_stop_logging + 556899
6   libMicrosoft.CognitiveServices.Spee 0x000000011c31ed55 file_logger_stop_logging + 556261
7   libMicrosoft.CognitiveServices.Spee 0x000000011c356e73 file_logger_stop_logging + 785923
8   libMicrosoft.CognitiveServices.Spee 0x000000011c35623c file_logger_stop_logging + 782796
9   libMicrosoft.CognitiveServices.Spee 0x000000011c354202 file_logger_stop_logging + 774546
10  libMicrosoft.CognitiveServices.Spee 0x000000011c354423 file_logger_stop_logging + 775091
11  libMicrosoft.CognitiveServices.Spee 0x000000011c242cb3 recognizer_create_speech_recognizer_from_config + 2755
12  libMicrosoft.CognitiveServices.Spee 0x000000